# MIDS W261 Spring 2016 Homework Week 3

Ron Cordell<br />
W261-4<br />
ron.cordell@ischool.berkeley.edu<br />
January 26, 2016

## HW3.0.

**What is a merge sort? Where is it used in Hadoop?**

A merge sort merges sorted lists into a single sorted list. The merge sort works by establishing a pointer to the beginning of each sorted list as well as a new "merge" list. The objects or keys in each list referenced by the pointers are compared and the chosen one moved or copied to the location indicated by the pointer of the merge list. The merge list pointer is advanced as is the pointer for the list from which the object was moved. This is repeated until all objects in all list have been moved or copied to the new merge list. The comparator function "chooses" the object from the source lists based on the rules coded into the comparator function such as the largest, the smallest, etc.

Hadoop uses a merge sort during the shuffle process when it takes output from multiple sources such as mappers or combiners and merges them into the sorted streams used by downstream processes.

**How is  a combiner function in the context of Hadoop? 
Give an example where it can be used and justify why it should be used in the context of this problem.**

A combiner function is a function that can be used by Hadoop anywhere between the mappers and producers to help eliminate network and data traffic, especially as part of the shuffle. A an example combiner function typically provides a partial aggregation point for data emitted from the mapper to reduce hotspots in the shuffle.

An example where a combiner can be used to good effect is in a word count scenario, where the mapper emits the word as key and a value of 1. A combiner can perform aggregations on the key-value pairs by combining those with the same key and adding their values. This greatly reduces the granularity of the data required to shuffle and sort and provide to the reducers and helps reduce the amount of network and disk traffic of the shuffle, decreasing the overall run time.

**What is the Hadoop shuffle?**

![MapReduce Workflow](MapReduceWorkflow.png)

The Hadoop shuffle takes the mapper outpus, merges them and sorts them, computes a hash to partition them, and routes each partition to a reducer. It corresponds the the shuffle and sort section of the above diagram, between the mappers and reducers.

## HW3.1 Use Counters to do EDA (exploratory data analysis and to monitor progress)**

**The consumer complaints dataset consists of diverse consumer complaints, which have been reported across the United States regarding various types of loans. The dataset consists of records of the form:**

    Complaint ID,Product,Sub-product,Issue,Sub-issue,State,ZIP code,Submitted via,Date received,Date sent to company,Company,Company response,Timely response?,Consumer disputed?

**Here’s is the first few lines of the  of the Consumer Complaints  Dataset:**

    Complaint ID,Product,Sub-product,Issue,Sub-issue,State,ZIP code,Submitted via,Date received,Date sent to company,Company,Company response,Timely response?,Consumer disputed?
    1114245,Debt collection,Medical,Disclosure verification of debt,Not given enough info to verify debt,FL,32219,Web,11/13/2014,11/13/2014,"Choice Recovery, Inc.",Closed with explanation,Yes,
    1114488,Debt collection,Medical,Disclosure verification of debt,Right to dispute notice not received,TX,75006,Web,11/13/2014,11/13/2014,"Expert Global Solutions, Inc.",In progress,Yes,
    1114255,Bank account or service,Checking account,Deposits and withdrawals,,NY,11102,Web,11/13/2014,11/13/2014,"FNIS (Fidelity National Information Services, Inc.)",In progress,Yes,
    1115106,Debt collection,"Other (phone, health club, etc.)",Communication tactics,Frequent or repeated calls,GA,31721,Web,11/13/2014,11/13/2014,"Expert Global Solutions, Inc.",In progress,Yes,

**Now, let’s use Hadoop Counters to identify the number of complaints pertaining to debt collection, mortgage and other categories (all other categories get lumped into this one) in the consumer complaints dataset. Basically produce the distribution of the Product column in this dataset using counters (limited to 3 counters here).**

### Map

In [57]:
%%writefile mapper.py
#!/usr/bin/env python
#
# W261 HW 3.1 MapReduce and Counters
#
# Read from a CSV file of consumer complaints with fields as follows:
#
# Complaint ID,Product,Sub-product,Issue,Sub-issue,State,ZIP code,Submitted via,Date received,
# Date sent to company,Company,Company response,Timely response?,Consumer disputed?
#
# Use counters to count the number of complaints for Product Id's of debt collection, mortgage,
# and everything else (3 categories)
#
# Remember that in Hadoop streaming, to update a counter is to write to STDERR in the format
# reporter:counter:<group>,<counter>,<amount>

import sys

# Read data from STDIN and use counters to count the data
def main(separator=','):
    # input comes from STDIN (standard input) as the fields from the CSV
    for line in (sys.stdin):
        fields = line.split(separator)
        try:
            # check to see if this is a header by trying to convert the first field to an integer
            id = int(fields[0])
            # we have a real record, so do some mapping
            counter_name = None
            if (fields[1].lower() == 'debt collection' or \
                fields[1].lower() == 'mortgage'):
                counter_name = fields[1].strip().lower()
            else:
                counter_name = 'other'
            # update the counter
            sys.stderr.write("reporter:counter:Category Counters,{0},1\n".format(counter_name))
        except:
            # must be a header record so skip it
            pass

            

if __name__ == "__main__":
    main()


Overwriting mapper.py


In [40]:
# Set the appropriate execution mode on the file so we can execute it
!chmod a+x mapper.py

In [62]:
!/usr/local/Cellar/hadoop/2.7.1/sbin/start-yarn.sh
!/usr/local/Cellar/hadoop/2.7.1/sbin/start-dfs.sh
!/usr/local/Cellar/hadoop/2.7.1/sbin/./mr-jobhistory-daemon.sh \
    --config /usr/local/Cellar/hadoop/2.7.1/libexec/etc/hadoop/ start historyserver

starting yarn daemons
starting resourcemanager, logging to /usr/local/Cellar/hadoop/2.7.1/libexec/logs/yarn-rcordell-resourcemanager-Rons-iMac-Retina.local.out
localhost: starting nodemanager, logging to /usr/local/Cellar/hadoop/2.7.1/libexec/logs/yarn-rcordell-nodemanager-Rons-iMac-Retina.local.out
16/01/29 23:50:29 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Starting namenodes on [localhost]
localhost: starting namenode, logging to /usr/local/Cellar/hadoop/2.7.1/libexec/logs/hadoop-rcordell-namenode-Rons-iMac-Retina.local.out
localhost: starting datanode, logging to /usr/local/Cellar/hadoop/2.7.1/libexec/logs/hadoop-rcordell-datanode-Rons-iMac-Retina.local.out
Starting secondary namenodes [0.0.0.0]
0.0.0.0: starting secondarynamenode, logging to /usr/local/Cellar/hadoop/2.7.1/libexec/logs/hadoop-rcordell-secondarynamenode-Rons-iMac-Retina.local.out
16/01/29 23:50:43 WARN util.NativeCodeLoader: Unabl

In [76]:
!hdfs dfs -mkdir -p /user/rcordell
!hdfs dfs -put Consumer_Complaints.csv /user/rcordell
!hdfs dfs -put short_complaints.csv /user/rcordell

16/01/30 09:43:31 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
16/01/30 09:43:32 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
16/01/30 09:43:34 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [74]:
!hdfs dfs -rm -r /user/rcordell/recordsOutput

16/01/30 09:43:16 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
rm: `/user/rcordell/recordsOutput': No such file or directory


In [77]:
!yarn jar /usr/local/Cellar/hadoop/2.7.1/libexec/share/hadoop/tools/lib/hadoop-streaming-2.7.1.jar \
    -D mapred.reduce.tasks=0 \
    -file mapper.py \
    -mapper mapper.py \
    -input Consumer_Complaints.csv \
    -output recordsOutput

16/01/30 09:44:16 WARN streaming.StreamJob: -file option is deprecated, please use generic option -files instead.
16/01/30 09:44:16 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
packageJobJar: [mapper.py, /var/folders/z_/rfp5q2cd6db13d19v6yw0n8w0000gn/T/hadoop-unjar6059806740624592914/] [] /var/folders/z_/rfp5q2cd6db13d19v6yw0n8w0000gn/T/streamjob8735250773221487987.jar tmpDir=null
16/01/30 09:44:17 INFO client.RMProxy: Connecting to ResourceManager at /0.0.0.0:8032
16/01/30 09:44:17 INFO client.RMProxy: Connecting to ResourceManager at /0.0.0.0:8032
16/01/30 09:44:17 INFO mapred.FileInputFormat: Total input paths to process : 1
16/01/30 09:44:17 INFO mapreduce.JobSubmitter: number of splits:2
16/01/30 09:44:17 INFO Configuration.deprecation: mapred.reduce.tasks is deprecated. Instead, use mapreduce.job.reduces
16/01/30 09:44:17 INFO mapreduce.JobSubmitter: Submitting tokens for job: job_1454175435207_0

In [73]:
!/usr/local/Cellar/hadoop/2.7.1/sbin/stop-yarn.sh
!/usr/local/Cellar/hadoop/2.7.1/sbin/stop-dfs.sh
!/usr/local/Cellar/hadoop/2.7.1/sbin/./mr-jobhistory-daemon.sh \
    --config /usr/local/Cellar/hadoop/2.7.1/libexec/etc/hadoop/ stop historyserver

stopping yarn daemons
stopping resourcemanager
localhost: stopping nodemanager
localhost: nodemanager did not stop gracefully after 5 seconds: killing with kill -9
no proxyserver to stop
16/01/30 00:34:17 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Stopping namenodes on [localhost]
localhost: stopping namenode
localhost: stopping datanode
Stopping secondary namenodes [0.0.0.0]
0.0.0.0: stopping secondarynamenode
16/01/30 00:34:34 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
no historyserver to stop


**Hadoop offers Job Tracker, an UI tool to determine the status and statistics of all jobs. Using the job tracker UI, developers can view the Counters that have been created. Screenshot your  job tracker UI as your job completes and include it here. Make sure that your user defined counters are visible. **

![Job Tracker](jobtrackercounters.png)

## HW 3.2 Analyze the performance of your Mappers, Combiners and Reducers using Counters

For this brief study the Input file will be one record (the next line only): 
foo foo quux labs foo bar quux


Perform a word count analysis of this single record dataset using a Mapper and Reducer based WordCount (i.e., no combiners are used here) using user defined Counters to count up how many time the mapper and reducer are called. What is the value of your user defined Mapper Counter, and Reducer Counter after completing this word count job. The answer  should be 1 and 4 respectively. Please explain.

In [113]:
%%writefile mapper32.py
#!/usr/bin/env python
#
# W261 HW 3.2 MapReduce and Counters for Code Analysis
#
# Simple word counter
#
# Use counters to count the number of times the mapper is called
#
# Remember that in Hadoop streaming, to update a counter is to write to STDERR in the format
# reporter:counter:<group>,<counter>,<amount>

import sys

def read_input(file):
    for line in file:
        # split the line into words
        yield line.split()
        
# Read data from STDIN and use counters to count the data
def main(separator=','):    
    data = read_input(sys.stdin)
    for words in data:
        # write the results to STDOUT
        # tab-delimited; the trivial word count is 1
        for word in words:
            sys.stdout.write('{0}{1}{2}\n'.format(word, separator, 1))         

if __name__ == "__main__":
    # increment counter for mapper call, write to STDERR
    sys.stderr.write("reporter:counter:Code Call Counters,mapper,1\n")
    main()

Overwriting mapper32.py


In [114]:
%%writefile reducer32.py
#!/usr/bin/env python

#
# Use counters to count the number of times the reducer is called
#
from itertools import groupby
from operator import itemgetter
import sys

def read(file, separator='\t'):
    for line in file:
        yield line.rstrip().split(separator, 1)

def main(separator='\t'):    
    # input comes from STDIN (standard input)
    data = read(sys.stdin, separator=separator)
    # groupby groups multiple word-count pairs by word,
    # and creates an iterator that returns consecutive keys and their group:
    #   current_word - string containing a word (the key)
    #   group - iterator yielding all ["&lt;current_word&gt;", "&lt;count&gt;"] items
    for current_word, group in groupby(data, itemgetter(0)):
        try:
            total_count = sum(int(count) for current_word, count in group)
            sys.stdout.write("{0}{1}{2}".format(current_word, separator, total_count))
        except ValueError:
            # count was not a number, so silently discard this item
            pass

if __name__ == "__main__":
    # increment counter for reducer call, write to STDERR
    sys.stderr.write("reporter:counter:Code Call Counters,reducer,1\n")
    main()

Overwriting reducer32.py


In [84]:
!echo "foo foo quux labs foo bar quux" >foo.txt
!hdfs dfs -put foo.txt /user/rcordell

16/01/30 10:01:53 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [117]:
!hdfs dfs -rm -r /user/rcordell/recordsOutput

16/01/30 17:20:09 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
16/01/30 17:20:10 INFO fs.TrashPolicyDefault: Namenode trash configuration: Deletion interval = 0 minutes, Emptier interval = 0 minutes.
Deleted /user/rcordell/recordsOutput


In [118]:
!yarn jar /usr/local/Cellar/hadoop/2.7.1/libexec/share/hadoop/tools/lib/hadoop-streaming-2.7.1.jar \
    -D mapreduce.job.reduces=4 \
    -D mapreduce.job.maps=1 \
    -files "mapper32.py,reducer32.py" \
    -mapper mapper32.py \
    -reducer reducer32.py \
    -input foo.txt \
    -output recordsOutput

16/01/30 17:20:14 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
packageJobJar: [/var/folders/z_/rfp5q2cd6db13d19v6yw0n8w0000gn/T/hadoop-unjar8517455371226977358/] [] /var/folders/z_/rfp5q2cd6db13d19v6yw0n8w0000gn/T/streamjob5581936008517874544.jar tmpDir=null
16/01/30 17:20:15 INFO client.RMProxy: Connecting to ResourceManager at /0.0.0.0:8032
16/01/30 17:20:15 INFO client.RMProxy: Connecting to ResourceManager at /0.0.0.0:8032
16/01/30 17:20:16 INFO mapred.FileInputFormat: Total input paths to process : 1
16/01/30 17:20:16 INFO mapreduce.JobSubmitter: number of splits:1
16/01/30 17:20:16 INFO mapreduce.JobSubmitter: Submitting tokens for job: job_1454175435207_0011
16/01/30 17:20:16 INFO impl.YarnClientImpl: Submitted application application_1454175435207_0011
16/01/30 17:20:16 INFO mapreduce.Job: The url to track the job: http://Rons-iMac-Retina.local:8088/proxy/application_1454175435207_0011/
16/01/3

Please use mulitple mappers and reducers for these jobs (at least 2 mappers and 2 reducers).
Perform a word count analysis of the Issue column of the Consumer Complaints  Dataset using a Mapper and Reducer based WordCount (i.e., no combiners used anywhere)  using user defined Counters to count up how many time the mapper and reducer are called. What is the value of your user defined Mapper Counter, and Reducer Counter after completing your word count job. 

In [182]:
%%writefile mapper32b.py
#!/usr/bin/env python
#
# W261 HW 3.2 MapReduce and Counters for Code Analysis
# Read from a CSV file of consumer complaints with fields as follows:
#
# Complaint ID,Product,Sub-product,Issue,Sub-issue,State,ZIP code,Submitted via,Date received,
# Date sent to company,Company,Company response,Timely response?,Consumer disputed?
#
# Use counters to count the number of times the mapper is called
#
# Remember that in Hadoop streaming, to update a counter is to write to STDERR in the format
# reporter:counter:<group>,<counter>,<amount>

import sys
import re

WORD_RE = re.compile(r"[\w']+")

# Read data from STDIN and use counters to count the data
def main(separator='\t'):   
    for line in sys.stdin:
        fields = line.split(',')
        try:
            # check to see if this is a header by trying to convert the first field to an integer
            id = int(fields[0])
            # we have a real record, so do some mapping
            for word in WORD_RE.findall(fields[3]):
                sys.stdout.write('{0}{1}{2}\n'.format(word, separator, 1))
        except:
            # must be a header record so skip it
            pass

if __name__ == "__main__":
    # increment counter for mapper call, write to STDERR
    sys.stderr.write("reporter:counter:Code Call Counters,mapper,1\n")
    main()

Overwriting mapper32b.py


In [194]:
%%writefile reducer32b.py
#!/usr/bin/env python

from itertools import groupby
from operator import itemgetter
import sys

def read(file, separator='\t'):
    for line in file:
        yield line.rstrip().split(separator, 1)

def main(separator='\t'):
    # input comes from STDIN (standard input)
    data = read(sys.stdin, separator=separator)
    # groupby groups multiple word-count pairs by word,
    # and creates an iterator that returns consecutive keys and their group:
    #   current_word - string containing a word (the key)
    #   group - iterator yielding all ["&lt;current_word&gt;", "&lt;count&gt;"] items
    for current_word, group in groupby(data, itemgetter(0)):
        try:
            total_count = sum(int(count) for current_word, count in group)
            sys.stderr.write("reporter:counter:Code Call Counters,reducer pairs,1\n")
            sys.stdout.write("{0}{1}{2}\n".format(current_word, separator, total_count))
        except ValueError:
            sys.stderr.write("reporter:counter:Code Call Counters,reducer skipped pairs,1\n")
            # count was not a number, so silently discard this item
            pass

if __name__ == "__main__":
    # increment counter for reducer call, write to STDERR
    sys.stderr.write("reporter:counter:Code Call Counters,reducer,1\n")
    main()

Overwriting reducer32b.py


In [184]:
!chmod a+x mapper32b.py
!chmod a+x reducer32b.py

In [ ]:
!cat short_complaints.csv | ./mapper32b.py | sort -k1,1 | ./reducer32b.py

In [195]:
!hdfs dfs -rm -r /user/rcordell/recordsOutput

16/01/30 19:16:41 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
16/01/30 19:16:41 INFO fs.TrashPolicyDefault: Namenode trash configuration: Deletion interval = 0 minutes, Emptier interval = 0 minutes.
Deleted /user/rcordell/recordsOutput


In [196]:
!yarn jar /usr/local/Cellar/hadoop/2.7.1/libexec/share/hadoop/tools/lib/hadoop-streaming-2.7.1.jar \
    -D mapreduce.job.reduces=2 \
    -D mapreduce.job.maps=2 \
    -files "mapper32b.py,reducer32b.py" \
    -mapper mapper32b.py \
    -reducer reducer32b.py \
    -input Consumer_Complaints.csv \
    -output recordsOutput

16/01/30 19:16:43 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
packageJobJar: [/var/folders/z_/rfp5q2cd6db13d19v6yw0n8w0000gn/T/hadoop-unjar3393828723429341869/] [] /var/folders/z_/rfp5q2cd6db13d19v6yw0n8w0000gn/T/streamjob6639340843874969134.jar tmpDir=null
16/01/30 19:16:44 INFO client.RMProxy: Connecting to ResourceManager at /0.0.0.0:8032
16/01/30 19:16:44 INFO client.RMProxy: Connecting to ResourceManager at /0.0.0.0:8032
16/01/30 19:16:44 INFO mapred.FileInputFormat: Total input paths to process : 1
16/01/30 19:16:44 INFO mapreduce.JobSubmitter: number of splits:2
16/01/30 19:16:45 INFO mapreduce.JobSubmitter: Submitting tokens for job: job_1454175435207_0018
16/01/30 19:16:45 INFO impl.YarnClientImpl: Submitted application application_1454175435207_0018
16/01/30 19:16:45 INFO mapreduce.Job: The url to track the job: http://Rons-iMac-Retina.local:8088/proxy/application_1454175435207_0018/
16/01/3

In [197]:
!hdfs dfs -cat /user/rcordell/recordsOutput/part-00000 | wc -l

16/01/30 19:17:08 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
      86


Perform a word count analysis of the Issue column of the Consumer Complaints  Dataset using a Mapper, Reducer, and standalone combiner (i.e., not an in-memory combiner) based WordCount using user defined Counters to count up how many time the mapper, combiner, reducer are called. What is the value of your user defined Mapper Counter, and Reducer Counter after completing your word count job. 

In [100]:
%%writefile combinerQ2part3.py
#!/usr/bin/env python

from itertools import groupby
from operator import itemgetter
import sys

def read(file, separator='\t'):
    for line in file:
        yield line.rstrip().split(separator, 1)

def main(separator='\t'):
    # input comes from STDIN (standard input)
    data = read(sys.stdin, separator=separator)
    # groupby groups multiple word-count pairs by word,
    # and creates an iterator that returns consecutive keys and their group:
    #   current_word - string containing a word (the key)
    #   group - iterator yielding all ["&lt;current_word&gt;", "&lt;count&gt;"] items
    for current_word, group in groupby(data, itemgetter(0)):
        try:
            total_count = sum(int(count) for current_word, count in group)
            sys.stderr.write("reporter:counter:Code Call Counters,combiner pairs,1\n")
            sys.stdout.write("{0}{1}{2}\n".format(current_word, separator, total_count))
        except ValueError:
            sys.stderr.write("reporter:counter:Code Call Counters,combiner skipped pairs,1\n")            
            # count was not a number, so silently discard this item
            pass

if __name__ == "__main__":
    # increment counter for combiner call, write to STDERR
    sys.stderr.write("reporter:counter:Code Call Counters,combiner,1\n")
    main()

Writing combinerQ2part3.py


In [101]:
!chmod a+x combinerQ2part3.py

In [104]:
!hdfs dfs -rm -r /user/rcordell/recordsOutput

16/01/31 14:02:02 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
16/01/31 14:02:02 INFO fs.TrashPolicyDefault: Namenode trash configuration: Deletion interval = 0 minutes, Emptier interval = 0 minutes.
Deleted /user/rcordell/recordsOutput


In [105]:
!yarn jar /usr/local/Cellar/hadoop/2.7.1/libexec/share/hadoop/tools/lib/hadoop-streaming-2.7.1.jar \
    -D mapreduce.job.reduces=2 \
    -D mapreduce.job.maps=2 \
    -files "mapper32b.py,combinerQ2part3.py,reducer32b.py" \
    -mapper mapper32b.py \
    -reducer reducer32b.py \
    -combiner combinerQ2part3.py \
    -input Consumer_Complaints.csv \
    -output recordsOutput

16/01/31 14:02:03 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
packageJobJar: [/var/folders/z_/rfp5q2cd6db13d19v6yw0n8w0000gn/T/hadoop-unjar6847100501139577256/] [] /var/folders/z_/rfp5q2cd6db13d19v6yw0n8w0000gn/T/streamjob7429796512504517614.jar tmpDir=null
16/01/31 14:02:03 INFO client.RMProxy: Connecting to ResourceManager at /0.0.0.0:8032
16/01/31 14:02:03 INFO client.RMProxy: Connecting to ResourceManager at /0.0.0.0:8032
16/01/31 14:02:04 INFO mapred.FileInputFormat: Total input paths to process : 1
16/01/31 14:02:04 INFO mapreduce.JobSubmitter: number of splits:2
16/01/31 14:02:04 INFO mapreduce.JobSubmitter: Submitting tokens for job: job_1454175435207_0083
16/01/31 14:02:04 INFO impl.YarnClientImpl: Submitted application application_1454175435207_0083
16/01/31 14:02:04 INFO mapreduce.Job: The url to track the job: http://Rons-iMac-Retina.local:8088/proxy/application_1454175435207_0083/
16/01/3

In [106]:
!hdfs dfs -cat /user/rcordell/recordsOutput/part-00000

16/01/31 14:03:01 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
APR	3431
Account	16555
Applied	139
Arbitration	168
Bankruptcy	222
Billing	8158
Can't	1999
Cash	240
Closing	2795
Cont'd	11848
Convenience	75
Credit	14768
Debt	1343
Delinquent	1061
Deposits	10555
Disclosure	5214
False	2508
I	925
Incorrect	29133
Making	3226
Overlimit	127
Payoff	1155
Received	118
Repaying	3844
Sale	139
Settlement	4350
Unable	4357
Unsolicited	640
Workout	350
Wrong	98
a	3503
account	4126
acct	163
an	2505
and	16448
available	274
being	5663
by	5663
caused	5663
changes	350
charges	131
checks	75
company's	4858
credit	40483
debt	17966
delay	243
determination	1490
did	139
disputes	6938
expect	807
fees	807
for	929
issuance	640
issue	1098
not	12353
of	10885
on	29069
or	22533
owed	11848
payments	3226
process	5505
processing	243
promised	274
protection	4139
receive	139
received	98
relations	1367
repay	1647
representation	2508
service	1518


### Part 4
Using a single reducer: What are the top 50 most frequent terms in your word count analysis? Present the top 50 terms and their frequency and their relative frequency. Present the top 50 terms and their frequency and their relative frequency. If there are ties please sort the tokens in alphanumeric/string order. Present bottom 10 tokens (least frequent items). 

In [107]:
%%writefile mapperQ2part4.py
#!/usr/bin/env python
#
# W261 HW 3.2 MapReduce and Counters for Code Analysis
# Read from a CSV file of consumer complaints with fields as follows:
#
# Complaint ID,Product,Sub-product,Issue,Sub-issue,State,ZIP code,Submitted via,Date received,
# Date sent to company,Company,Company response,Timely response?,Consumer disputed?
#
# Use counters to count the number of times the mapper is called
#
# Remember that in Hadoop streaming, to update a counter is to write to STDERR in the format
# reporter:counter:<group>,<counter>,<amount>

import sys
import re

WORD_RE = re.compile(r"[\w']+")

# Read data from STDIN and use counters to count the data
def main(separator='\t'):   
    for line in sys.stdin:
        fields = line.split(',')
        try:
            # check to see if this is a header by trying to convert the first field to an integer
            id = int(fields[0])
            # we have a real record, so do some mapping
            for word in WORD_RE.findall(fields[3]):
                sys.stdout.write('{0}{1}{2}\n'.format(word.lower(), separator, 1))
                sys.stdout.write('{0}{1}{2}\n'.format('*',  separator, 1))
        except:
            # must be a header record so skip it
            pass

if __name__ == "__main__":
    # increment counter for mapper call, write to STDERR
    sys.stderr.write("reporter:counter:Code Call Counters,mapper,1\n")
    main()

Writing mapperQ2part4.py


In [108]:
!chmod a+x mapperQ2part4.py

In [109]:
%%writefile combinerQ2part4.py
#!/usr/bin/env python

from itertools import groupby
from operator import itemgetter
import sys

def read(file, separator='\t'):
    for line in file:
        yield line.rstrip().split(separator, 1)

def main(separator='\t'):
    # input comes from STDIN (standard input)
    data = read(sys.stdin, separator=separator)
    # groupby groups multiple word-count pairs by word,
    # and creates an iterator that returns consecutive keys and their group:
    #   current_word - string containing a word (the key)
    #   group - iterator yielding all ["&lt;current_word&gt;", "&lt;count&gt;"] items
    for current_word, group in groupby(data, itemgetter(0)):
        try:
            total_count = sum(int(count) for current_word, count in group)
            sys.stderr.write("reporter:counter:Code Call Counters,combiner pairs,1\n")
            sys.stdout.write("{0}{1}{2}\n".format(current_word, separator, total_count))
            if current_word == '*':
                sys.stderr.write("reporter:counter:Code Call Counters,combiner total flags,1\n")
        except ValueError:
            sys.stderr.write("reporter:counter:Code Call Counters,combiner skipped pairs,1\n")            
            # count was not a number, so silently discard this item
            pass

if __name__ == "__main__":
    # increment counter for combiner call, write to STDERR
    sys.stderr.write("reporter:counter:Code Call Counters,combiner,1\n")
    main()

Writing combinerQ2part4.py


In [110]:
!chmod a+x combinerQ2part4.py

In [111]:
%%writefile reducerQ2part4.py
#!/usr/bin/env python

from itertools import groupby
from operator import itemgetter
import sys

def read(file, separator='\t'):
    for line in file:
        yield line.rstrip().split(separator, 1)

def main(separator='\t'):
    total = 1
    total_first = True
    
    # input comes from STDIN (standard input)
    data = read(sys.stdin, separator=separator)
    # groupby groups multiple word-count pairs by word,
    # and creates an iterator that returns consecutive keys and their group:
    #   current_word - string containing a word (the key)
    #   group - iterator yielding all ["&lt;current_word&gt;", "&lt;count&gt;"] items
    for current_word, group in groupby(data, itemgetter(0)):
        try:
            total_count = sum(int(count) for current_word, count in group)
            if current_word == '*':
                total = total_count
                sys.stderr.write("reporter:counter:Code Call Counters,reducer total indicators,1\n")
                sys.stderr.write("reporter:counter:Code Call Counters,reducer word count,{0}\n".format(total))
                if total_first:
                    sys.stderr.write("reporter:counter:Code Call Counters,reducer recvd total first,1\n")
            else:
                sys.stderr.write("reporter:counter:Code Call Counters,reducer processed,1\n")
                sys.stdout.write("{0:20}\t{1:10}\t{2}\n".format(current_word, total_count, 
                                                              float(total_count)/float(total))) 
                total_first = False
                
        except ValueError:
            sys.stderr.write("reporter:counter:Code Call Counters,reducer skipped pairs,1\n")            
            # count was not a number, so silently discard this item
            pass

if __name__ == "__main__":
    # increment counter for combiner call, write to STDERR
    sys.stderr.write("reporter:counter:Code Call Counters,reducer,1\n")
    main()

Writing reducerQ2part4.py


In [112]:
!chmod a+x reducerQ2part4.py

In [113]:
!hdfs dfs -rm -r /user/rcordell/recordsOutput

16/01/31 14:04:37 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
16/01/31 14:04:37 INFO fs.TrashPolicyDefault: Namenode trash configuration: Deletion interval = 0 minutes, Emptier interval = 0 minutes.
Deleted /user/rcordell/recordsOutput


In [114]:
# mapreduce.partition.keycomparator.options -> partition on which part of the key
# stream.num.map.output.key.fields=2 -> tells the partitioner to treat the output of the mapper
#       as a key with 2 fields and no value
# mapreduce.partition.keycomparator.options="-k2nr -k1" -> sort first on the second key field in numeric descending
#       then sort on the first key field as alpha ascending

!yarn jar /usr/local/Cellar/hadoop/2.7.1/libexec/share/hadoop/tools/lib/hadoop-streaming-2.7.1.jar \
    -D stream.num.map.output.key.fields=2 \
    -D mapreduce.partition.keypartitioner.options="-k2,2nr" \
    -D mapreduce.job.output.key.comparator.class=org.apache.hadoop.mapred.lib.KeyFieldBasedComparator \
    -D mapreduce.partition.keycomparator.options="-k2nr -k1" \
    -D mapreduce.job.reduces=1 \
    -D mapreduce.job.maps=2 \
    -files "mapperQ2part4.py,reducerQ2part4.py" \
    -mapper mapperQ2part4.py \
    -reducer reducerQ2part4.py \
    -input Consumer_Complaints.csv \
    -output recordsOutput

16/01/31 14:04:39 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
packageJobJar: [/var/folders/z_/rfp5q2cd6db13d19v6yw0n8w0000gn/T/hadoop-unjar3426155122428197504/] [] /var/folders/z_/rfp5q2cd6db13d19v6yw0n8w0000gn/T/streamjob7024171405453559567.jar tmpDir=null
16/01/31 14:04:39 INFO client.RMProxy: Connecting to ResourceManager at /0.0.0.0:8032
16/01/31 14:04:39 INFO client.RMProxy: Connecting to ResourceManager at /0.0.0.0:8032
16/01/31 14:04:40 INFO mapred.FileInputFormat: Total input paths to process : 1
16/01/31 14:04:40 INFO mapreduce.JobSubmitter: number of splits:2
16/01/31 14:04:40 INFO mapreduce.JobSubmitter: Submitting tokens for job: job_1454175435207_0084
16/01/31 14:04:40 INFO impl.YarnClientImpl: Submitted application application_1454175435207_0084
16/01/31 14:04:40 INFO mapreduce.Job: The url to track the job: http://Rons-iMac-Retina.local:8088/proxy/application_1454175435207_0084/
16/01/3

** Top 50 Occuring Words with Count and Frequency**

In [115]:
!hdfs dfs -cat /user/rcordell/recordsOutput/part-00000 | sort -k2,2nr | head -50

16/01/31 14:05:08 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
loan                	    119630	0.122011418874
modification        	     70487	0.0718901519865
credit              	     55251	0.0563508560076
servicing           	     36767	0.0374989036005
report              	     34903	0.0355977978178
incorrect           	     29133	0.0297129371064
information         	     29069	0.0296476630882
on                  	     29069	0.0296476630882
or                  	     22533	0.0229815539704
account             	     20681	0.0210926870662
debt                	     19309	0.0196933752991
and                 	     16448	0.0167754227003
opening             	     16205	0.0165275854121
club                	     12545	0.0127947274912
health              	     12545	0.0127947274912
not                 	     12353	0.0125989054363
attempts            	     11848	0.0120838526357
collect             	     11848	0.012

** Bottom 10 Occuring Words with Count and Frequency **

In [116]:
!hdfs dfs -cat /user/rcordell/recordsOutput/part-00000 | sort -k2,2nr | tail -10

16/01/31 14:05:14 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
apply               	       118	0.00012034897122
amount              	        98	9.9950840505e-05
credited            	        92	9.38314012904e-05
payment             	        92	9.38314012904e-05
checks              	        75	7.64929901824e-05
convenience         	        75	7.64929901824e-05
amt                 	        71	7.24133640393e-05
day                 	        71	7.24133640393e-05
disclosures         	        64	6.5274018289e-05
missing             	        64	6.5274018289e-05


### 3.2.1 OPTIONAL 
Using 2 reducers: What are the top 50 most frequent terms in your word count analysis? Present the top 50 terms and their frequency and their relative frequency. Present the top 50 terms and their frequency and their relative frequency. If there are ties please sort the tokens in alphanumeric/string order. Present bottom 10 tokens (least frequent items). 

In [117]:
!hdfs dfs -rm -r /user/rcordell/recordsOutput

16/01/31 14:05:40 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
16/01/31 14:05:40 INFO fs.TrashPolicyDefault: Namenode trash configuration: Deletion interval = 0 minutes, Emptier interval = 0 minutes.
Deleted /user/rcordell/recordsOutput


In [118]:
# mapreduce.partition.keycomparator.options -> partition on which part of the key
# stream.num.map.output.key.fields=2 -> tells the partitioner to treat the output of the mapper
#       as a key with 2 fields and no value
# mapreduce.partition.keycomparator.options="-k2nr -k1" -> sort first on the second key field in numeric descending
#       then sort on the first key field as alpha ascending

!yarn jar /usr/local/Cellar/hadoop/2.7.1/libexec/share/hadoop/tools/lib/hadoop-streaming-2.7.1.jar \
    -D stream.num.map.output.key.fields=2 \
    -D mapreduce.partition.keypartitioner.options="-k2,2nr" \
    -D mapreduce.job.output.key.comparator.class=org.apache.hadoop.mapred.lib.KeyFieldBasedComparator \
    -D mapreduce.partition.keycomparator.options="-k2nr -k1" \
    -D mapreduce.job.reduces=2 \
    -D mapreduce.job.maps=2 \
    -files "mapperQ2part4.py,reducerQ2part4.py" \
    -mapper mapperQ2part4.py \
    -reducer reducerQ2part4.py \
    -input Consumer_Complaints.csv \
    -output recordsOutput

16/01/31 14:05:43 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
packageJobJar: [/var/folders/z_/rfp5q2cd6db13d19v6yw0n8w0000gn/T/hadoop-unjar5153604838863109387/] [] /var/folders/z_/rfp5q2cd6db13d19v6yw0n8w0000gn/T/streamjob47573597575542206.jar tmpDir=null
16/01/31 14:05:43 INFO client.RMProxy: Connecting to ResourceManager at /0.0.0.0:8032
16/01/31 14:05:43 INFO client.RMProxy: Connecting to ResourceManager at /0.0.0.0:8032
16/01/31 14:05:44 INFO mapred.FileInputFormat: Total input paths to process : 1
16/01/31 14:05:44 INFO mapreduce.JobSubmitter: number of splits:2
16/01/31 14:05:44 INFO mapreduce.JobSubmitter: Submitting tokens for job: job_1454175435207_0085
16/01/31 14:05:44 INFO impl.YarnClientImpl: Submitted application application_1454175435207_0085
16/01/31 14:05:44 INFO mapreduce.Job: The url to track the job: http://Rons-iMac-Retina.local:8088/proxy/application_1454175435207_0085/
16/01/31 

** Needs more work - the word count totals are not getting to the reducers; only one of the them is getting it.**

May need more partitioning control

## HW3.3. Shopping Cart Analysis
Product Recommendations: The action or practice of selling additional products or services 
to existing customers is called cross-selling. Giving product recommendation is 
one of the examples of cross-selling that are frequently used by online retailers. 
One simple method to give product recommendations is to recommend products that are frequently
browsed together by the customers.

For this homework use the online browsing behavior dataset located at: 

       https://www.dropbox.com/s/zlfyiwa70poqg74/ProductPurchaseData.txt?dl=0

Each line in this dataset represents a browsing session of a customer. 
On each line, each string of 8 characters represents the id of an item browsed during that session. 
The items are separated by spaces.

Here are the first few lines of the ProductPurchaseData 

    FRO11987 ELE17451 ELE89019 SNA90258 GRO99222 
    GRO99222 GRO12298 FRO12685 ELE91550 SNA11465 ELE26917 ELE52966 FRO90334 SNA30755 ELE17451 FRO84225 SNA80192 
    ELE17451 GRO73461 DAI22896 SNA99873 FRO86643 
    ELE17451 ELE37798 FRO86643 GRO56989 ELE23393 SNA11465 
    ELE17451 SNA69641 FRO86643 FRO78087 SNA11465 GRO39357 ELE28573 ELE11375 DAI54444 


Do some exploratory data analysis of this dataset. 

How many unique items are available from this supplier?

Using a single reducer: Report your findings such as number of unique products; largest basket; report the top 50 most frequently purchased items,  their frequency,  and their relative frequency (break ties by sorting the products alphabetical order) etc. using Hadoop Map-Reduce. 

In [366]:
%%writefile mapperQ3part1.py
#!/usr/bin/env python
#
# W261 HW 3.3 MapReduce and Counters for Code Analysis
#
# Read a line of product ids where each line is a user session and the ids are products the user viewed
# Emit product_id, 1
#
# Use counters to count the number of times the mapper is called
#
# Remember that in Hadoop streaming, to update a counter is to write to STDERR in the format
# reporter:counter:<group>,<counter>,<amount>

import sys


# Read data from STDIN and use counters to count the data
def main(separator='\t'):   
    for line in sys.stdin:
        basket = line.split()
        for item in basket:
            # emit item, 1, item count in basket
            sys.stdout.write('{0}{1}{2}{1}{3}\n'.format(item, separator, 1, len(basket)))
            
            # emit *, 1, item count in basket
            sys.stdout.write('{0}{1}{2}{1}{3}\n'.format('*',  separator, 1, len(basket)))
        
if __name__ == "__main__":
    # increment counter for mapper call, write to STDERR
    sys.stderr.write("reporter:counter:Code Call Counters,mapper,1\n")
    main()

Overwriting mapperQ3part1.py


In [121]:
!chmod a+x mapperQ3part1.py

In [390]:
%%writefile combinerQ3part1.py
#!/usr/bin/env python

import sys

def main(separator='\t'):
    largest_basket = 0
    current_item = None
    current_count = 0
    
    # input comes from STDIN (standard input)
    for line in sys.stdin:
        # read a line and split into item, count, and basketsize
        item, count, basketsize = line.split(separator)

        # still counting the same key, keep accumulating
        if current_item == item.strip():
            current_count += int(count.strip())
        else:
            if current_item:
                # emit the accumulated key count
                sys.stdout.write("{0}{1}{2}{1}{3}\n".format(current_item, separator, current_count, largest_basket))

            # set/reset state
            current_count = int(count.strip())
            current_item = item.strip()
            largest_basket = max(largest_basket, int(basketsize.strip()))

    if current_item:
        # emit the last accumulated key count
        sys.stdout.write("{0}{1}{2}{1}{3}\n".format(current_item, separator, current_count, largest_basket))

    
if __name__ == "__main__":
    # increment counter for combiner call, write to STDERR
    sys.stderr.write("reporter:counter:Code Call Counters,combiner,1\n")
    main()

Overwriting combinerQ3part1.py


In [371]:
!chmod a+x combinerQ3part1.py

In [404]:
%%writefile reducerQ3part1.py
#!/usr/bin/env python

from collections import defaultdict
import sys

def main(separator='\t'):
    total = 0
    unique_items = 0
    largest_basket = 0
    current_item = None
    current_count = 0
    
    # input comes from STDIN (standard input)
    for line in sys.stdin:
        item, count, basketsize = line.split(separator)
        if item == '*':
            total += int(count.strip())
            sys.stderr.write("reporter:counter:Code Call Counters,Recieved Totals,1\n")
        else:
            # still counting the same key, keep accumulating
            if current_item == item.strip():
                current_count += int(count.strip())
            else:
                if current_item:
                    # emit the accumulated key count
                    sys.stdout.write("{0}{1}{2}\n".format(current_item, separator, current_count))

                # set/reset state
                current_count = int(count.strip())
                current_item = item.strip()
                largest_basket = max(largest_basket, int(basketsize.strip()))
                # increment the unique item count
                unique_items += 1
                # increment the unique item counter
                sys.stderr.write("reporter:counter:Code Call Counters,Unique Item Counter,1\n")

    if current_item:
            # emit the last accumulated key count
            sys.stdout.write("{0}{1}{2}\n".format(current_item, separator, current_count))
    
    # emit special key for max basket size
    sys.stdout.write("{0}{1}{2}\n".format('*MAXBASKET', separator, largest_basket))
    
    # emit special key for max basket size
    sys.stdout.write("{0}{1}{2}\n".format('*UNIQUEITEMS', separator, unique_items))
    
    #emit special key for total items
    sys.stdout.write("{0}{1}{2}\n".format('*TOTALITEMS', separator, total))    
    
if __name__ == "__main__":
    # increment counter for combiner call, write to STDERR
    sys.stderr.write("reporter:counter:Code Call Counters,reducer,1\n")
    main()

Overwriting reducerQ3part1.py


In [124]:
!chmod a+x reducerQ3part1.py

In [184]:
%%writefile reducerAQ3part1.py
#!/usr/bin/env python

from operator import itemgetter
import sys

def main(separator='\t'):
    total = 1
    total_first = True
    
    # input comes from STDIN (standard input)
    for line in (sys.stdin):
        item, count = line.split(separator)
        try:
            if item == '*':
                total = int(count)
                if total_first:
                    sys.stderr.write("reporter:counter:Code Call Counters,reducer saw total first,1\n")
                    sys.stderr.write("reporter:counter:Code Call Counters,unique items,1\total")
            else:
                sys.stderr.write("reporter:counter:Code Call Counters,reducer pairs,1\n")
                sys.stdout.write("{0}{1}{2}\n".format(item, separator, count))
                total_first = False

        except ValueError:
            sys.stderr.write("reporter:counter:Code Call Counters,reducer skipped pairs,1\n")            
            # count was not a number, so silently discard this item
            pass
        
if __name__ == "__main__":
    # increment counter for combiner call, write to STDERR
    sys.stderr.write("reporter:counter:Code Call Counters,reducer,1\n")
    main()

Overwriting reducerAQ3part1.py


In [187]:
!chmod a+x reducerAQ3part1.py

In [130]:
!hdfs dfs -put ~/Dropbox/W261/ProductPurchaseData.txt /user/rcordell

16/01/31 14:22:08 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
put: `/user/rcordell/ProductPurchaseData.txt': File exists


In [405]:
!hdfs dfs -rm -r /user/rcordell/recordsOutput

16/01/31 23:53:18 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
16/01/31 23:53:18 INFO fs.TrashPolicyDefault: Namenode trash configuration: Deletion interval = 0 minutes, Emptier interval = 0 minutes.
Deleted /user/rcordell/recordsOutput


In [406]:
# mapreduce.partition.keycomparator.options -> partition on which part of the key
# stream.num.map.output.key.fields=2 -> tells the partitioner to treat the output of the mapper
#       as a key with 2 fields and no value
# mapreduce.partition.keycomparator.options="-k2nr -k1" -> sort first on the second key field in numeric descending
#       then sort on the first key field as alpha ascending

!yarn jar /usr/local/Cellar/hadoop/2.7.1/libexec/share/hadoop/tools/lib/hadoop-streaming-2.7.1.jar \
    -D stream.num.map.output.key.fields=2 \
    -D mapred.text.key.partitioner.options="-k1,1" \
    -D mapred.output.key.comparator.class=org.apache.hadoop.mapred.lib.KeyFieldBasedComparator \
    -D mapred.text.key.comparator.options="-k1,1 -k2,2nr" \
    -D mapreduce.job.reduces=1 \
    -D mapreduce.job.maps=2 \
    -files "mapperQ3part1.py,combinerQ3part1.py,reducerQ3part1.py" \
    -mapper mapperQ3part1.py \
    -combiner combinerQ3part1.py \
    -reducer reducerQ3part1.py \
    -input ProductPurchaseData.txt \
    -output recordsOutput

16/01/31 23:53:19 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
packageJobJar: [/var/folders/z_/rfp5q2cd6db13d19v6yw0n8w0000gn/T/hadoop-unjar5190047823412462063/] [] /var/folders/z_/rfp5q2cd6db13d19v6yw0n8w0000gn/T/streamjob2633578674413320649.jar tmpDir=null
16/01/31 23:53:20 INFO client.RMProxy: Connecting to ResourceManager at /0.0.0.0:8032
16/01/31 23:53:20 INFO client.RMProxy: Connecting to ResourceManager at /0.0.0.0:8032
16/01/31 23:53:20 INFO mapred.FileInputFormat: Total input paths to process : 1
16/01/31 23:53:20 INFO mapreduce.JobSubmitter: number of splits:2
16/01/31 23:53:20 INFO Configuration.deprecation: mapred.text.key.comparator.options is deprecated. Instead, use mapreduce.partition.keycomparator.options
16/01/31 23:53:20 INFO Configuration.deprecation: mapred.text.key.partitioner.options is deprecated. Instead, use mapreduce.partition.keypartitioner.options
16/01/31 23:53:20 INFO Con

In [353]:
%%writefile mapperbQ3part1.py
#!/usr/bin/env python
#
# W261 HW 3.3 MapReduce and Counters for Code Analysis
#
# Read a line of product ids where each line is a user session and the ids are products the user viewed
# Emit product_id, 1
#
# Use counters to count the number of times the mapper is called
#
# Remember that in Hadoop streaming, to update a counter is to write to STDERR in the format
# reporter:counter:<group>,<counter>,<amount>

import sys

# Read data from STDIN and use counters to count the data
def main(separator='\t'):   
    for line in sys.stdin:
        item, count = line.strip().split(separator)
        sys.stdout.write('{0}{1}{2}\n'.format(item, separator, count))
        
if __name__ == "__main__":
    # increment counter for mapper call, write to STDERR
    sys.stderr.write("reporter:counter:Code Call Counters,mapper,1\n")
    main()

Overwriting mapperbQ3part1.py


In [291]:
!chmod a+x mapperbQ3part1.py

In [409]:
%%writefile reducerbQ3part1.py
#!/usr/bin/env python

import sys

total_items = 1.0
unique_items = 0
max_basket = 0

def main(separator='\t'):    
    # input comes from STDIN (standard input)
    for line in (sys.stdin):
        item, count = line.strip().split(separator)
        
        # these special keys should arrive first
        if item.strip() == '*TOTALITEMS':
            total_items = float(count.strip())
        elif item.strip() == '*UNIQUEITEMS':
            unique_items = int(count.strip())
        elif item.strip() == '*MAXBASKET':
            max_basket = int(count.strip())
        else:    
            sys.stdout.write('{0}{1}{2}{1}{3}\n'.format(item, separator, count, float(count)/total_items))
        
if __name__ == "__main__":
    # increment counter for combiner call, write to STDERR
    sys.stderr.write("reporter:counter:Code Call Counters,reducer,1\n")
    main()

Overwriting reducerbQ3part1.py


In [300]:
!chmod a+x reducerbQ3part1.py

In [410]:
!hdfs dfs -rm -r /user/rcordell/recordsOutput2

16/02/01 00:06:07 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
16/02/01 00:06:07 INFO fs.TrashPolicyDefault: Namenode trash configuration: Deletion interval = 0 minutes, Emptier interval = 0 minutes.
Deleted /user/rcordell/recordsOutput2


In [411]:
# mapreduce.partition.keycomparator.options -> partition on which part of the key
# stream.num.map.output.key.fields=2 -> tells the partitioner to treat the output of the mapper
#       as a key with 2 fields and no value
# mapreduce.partition.keycomparator.options="-k2nr -k1" -> sort first on the second key field in numeric descending
#       then sort on the first key field as alpha ascending

!yarn jar /usr/local/Cellar/hadoop/2.7.1/libexec/share/hadoop/tools/lib/hadoop-streaming-2.7.1.jar \
    -D stream.num.map.output.key.fields=2 \
    -D mapreduce.job.output.key.comparator.class=org.apache.hadoop.mapred.lib.KeyFieldBasedComparator \
    -D mapreduce.partition.keycomparator.options="-k2,2nr -k1,1" \
    -D mapreduce.job.reduces=1 \
    -D mapreduce.job.maps=1 \
    -files "mapperbQ3part1.py,reducerbQ3part1.py" \
    -mapper mapperbQ3part1.py \
    -reducer reducerbQ3part1.py \
    -input recordsOutput/part-00000 \
    -output recordsOutput2

16/02/01 00:06:09 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
packageJobJar: [/var/folders/z_/rfp5q2cd6db13d19v6yw0n8w0000gn/T/hadoop-unjar4708819701684813221/] [] /var/folders/z_/rfp5q2cd6db13d19v6yw0n8w0000gn/T/streamjob5995398753935075682.jar tmpDir=null
16/02/01 00:06:10 INFO client.RMProxy: Connecting to ResourceManager at /0.0.0.0:8032
16/02/01 00:06:10 INFO client.RMProxy: Connecting to ResourceManager at /0.0.0.0:8032
16/02/01 00:06:10 INFO mapred.FileInputFormat: Total input paths to process : 1
16/02/01 00:06:10 INFO mapreduce.JobSubmitter: number of splits:1
16/02/01 00:06:10 INFO mapreduce.JobSubmitter: Submitting tokens for job: job_1454175435207_0159
16/02/01 00:06:10 INFO impl.YarnClientImpl: Submitted application application_1454175435207_0159
16/02/01 00:06:10 INFO mapreduce.Job: The url to track the job: http://Rons-iMac-Retina.local:8088/proxy/application_1454175435207_0159/
16/02/0

In [412]:
!hdfs dfs -cat /user/rcordell/recordsOutput2/part-00000 | head -50

16/02/01 00:06:39 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
DAI62779	6667	0.0175067747831
FRO40251	3881	0.010191059387
ELE17451	3875	0.0101753040775
GRO73461	3602	0.00945843749344
SNA80324	3044	0.00799319370628
ELE32164	2851	0.0074863979161
DAI75645	2736	0.00718442114993
SNA45677	2455	0.0064465474865
FRO31317	2330	0.0061183118711
DAI85309	2293	0.00602115412894
ELE26917	2292	0.00601852824402
FRO80039	2233	0.00586360103355
GRO21487	2115	0.00555374661261
SNA99873	2083	0.00546971829507
GRO59710	2004	0.00526227338613
GRO71621	1920	0.00504169905258
FRO85978	1918	0.00503644728273
GRO30386	1840	0.00483162825872
ELE74009	1816	0.00476860702057
GRO56726	1784	0.00468457870302
DAI63921	1773	0.00465569396887
GRO46854	1756	0.00461105392517
ELE66600	1713	0.00449814087347
DAI83733	1712	0.00449551498855
FRO32293	1702	0.00446925613932
ELE66810	1697	0.0044561267147
SNA55762	1646	0.00432220658362
DAI22177	1627	0.0042723

### 3.3.1 OPTIONAL 
Using 2 reducers:  Report your findings such as number of unique products; largest basket; report the top 50 most frequently purchased items,  their frequency,  and their relative frequency (break ties by sorting the products alphabetical order) etc. using Hadoop Map-Reduce. 

## HW3.4. (Computationally prohibitive but then again Hadoop can handle this) Pairs

Suppose we want to recommend new products to the customer based on the products they
have already browsed on the online website. Write a map-reduce program 
to find products which are frequently browsed together. Fix the support count (cooccurence count) to s = 100 
(i.e. product pairs need to occur together at least 100 times to be considered frequent) 
and find pairs of items (sometimes referred to itemsets of size 2 in association rule mining) that have a support count of 100 or more.

List the top 50 product pairs with corresponding support count (aka frequency), and relative frequency or support (number of records where they coccur, the number of records where they coccur/the number of baskets in the dataset)  in decreasing order of support  for frequent (100>count) itemsets of size 2. 

Use the Pairs pattern (lecture 3)  to  extract these frequent itemsets of size 2. Free free to use combiners if they bring value. Instrument your code with counters for count the number of times your mapper, combiner and reducers are called.  

Please output records of the following form for the top 50 pairs (itemsets of size 2): 

      item1, item2, support count, support



Fix the ordering of the pairs lexicographically (left to right), 
and break ties in support (between pairs, if any exist) 
by taking the first ones in lexicographically increasing order. 

Report  the compute time for the Pairs job. Describe the computational setup used (E.g., single computer; dual core; linux, number of mappers, number of reducers)
Instrument your mapper, combiner, and reducer to count how many times each is called using Counters and report these counts.

In [366]:
%%writefile mapperQ4.py
#!/usr/bin/env python
#
# W261 HW 3.3 MapReduce and Counters for Code Analysis
#
# Read a line of product ids where each line is a user session and the ids are products the user viewed
# Emit product_id, 1
#
# Use counters to count the number of times the mapper is called
#
# Remember that in Hadoop streaming, to update a counter is to write to STDERR in the format
# reporter:counter:<group>,<counter>,<amount>

import sys


# Read data from STDIN and use counters to count the data
def main(separator='\t'):   
    for line in sys.stdin:
        basket = line.split()
        for item in basket:
            # emit item, 1, item count in basket
            sys.stdout.write('{0}{1}{2}{1}{3}\n'.format(item, separator, 1, len(basket)))
            
            # emit *, 1, item count in basket
            sys.stdout.write('{0}{1}{2}{1}{3}\n'.format('*',  separator, 1, len(basket)))
        
if __name__ == "__main__":
    # increment counter for mapper call, write to STDERR
    sys.stderr.write("reporter:counter:Code Call Counters,mapper,1\n")
    main()

Overwriting mapperQ3part1.py


In [121]:
!chmod a+x mapperQ4part1.py

In [390]:
%%writefile combinerQ4.py
#!/usr/bin/env python

import sys

def main(separator='\t'):
    largest_basket = 0
    current_item = None
    current_count = 0
    
    # input comes from STDIN (standard input)
    for line in sys.stdin:
        # read a line and split into item, count, and basketsize
        item, count, basketsize = line.split(separator)

        # still counting the same key, keep accumulating
        if current_item == item.strip():
            current_count += int(count.strip())
        else:
            if current_item:
                # emit the accumulated key count
                sys.stdout.write("{0}{1}{2}{1}{3}\n".format(current_item, separator, current_count, largest_basket))

            # set/reset state
            current_count = int(count.strip())
            current_item = item.strip()
            largest_basket = max(largest_basket, int(basketsize.strip()))

    if current_item:
        # emit the last accumulated key count
        sys.stdout.write("{0}{1}{2}{1}{3}\n".format(current_item, separator, current_count, largest_basket))

    
if __name__ == "__main__":
    # increment counter for combiner call, write to STDERR
    sys.stderr.write("reporter:counter:Code Call Counters,combiner,1\n")
    main()

Overwriting combinerQ3part1.py


In [371]:
!chmod a+x combinerQ4.py

In [ ]:
%%writefile reducerQ4.py
#!/usr/bin/env python

from collections import defaultdict
import sys

def main(separator='\t'):
    total = 0
    unique_items = 0
    largest_basket = 0
    current_item = None
    current_count = 0
    
    # input comes from STDIN (standard input)
    for line in sys.stdin:
        item, count, basketsize = line.split(separator)
        if item == '*':
            total += int(count.strip())
            sys.stderr.write("reporter:counter:Code Call Counters,Recieved Totals,1\n")
        else:
            # still counting the same key, keep accumulating
            if current_item == item.strip():
                current_count += int(count.strip())
            else:
                if current_item:
                    # emit the accumulated key count
                    sys.stdout.write("{0}{1}{2}\n".format(current_item, separator, current_count))

                # set/reset state
                current_count = int(count.strip())
                current_item = item.strip()
                largest_basket = max(largest_basket, int(basketsize.strip()))
                # increment the unique item count
                unique_items += 1
                # increment the unique item counter
                sys.stderr.write("reporter:counter:Code Call Counters,Unique Item Counter,1\n")

    if current_item:
            # emit the last accumulated key count
            sys.stdout.write("{0}{1}{2}\n".format(current_item, separator, current_count))
    
    # emit special key for max basket size
    sys.stdout.write("{0}{1}{2}\n".format('*MAXBASKET', separator, largest_basket))
    
    # emit special key for max basket size
    sys.stdout.write("{0}{1}{2}\n".format('*UNIQUEITEMS', separator, unique_items))
    
    #emit special key for total items
    sys.stdout.write("{0}{1}{2}\n".format('*TOTALITEMS', separator, total))    
    
if __name__ == "__main__":
    # increment counter for combiner call, write to STDERR
    sys.stderr.write("reporter:counter:Code Call Counters,reducer,1\n")
    main()

## HW3.5: Stripes
Repeat 3.4 using the stripes design pattern for finding cooccuring pairs.

Report  the compute times for stripes job versus the Pairs job. Describe the computational setup used (E.g., single computer; dual core; linux, number of mappers, number of reducers)

Instrument your mapper, combiner, and reducer to count how many times each is called using Counters and report these counts. Discuss the differences in these counts between the Pairs and Stripes jobs




OPTIONAL: all HW below this are optional 

NOTE:   -- as of 1/28/2016 the instructions needs to be completed (Talk to Jimi)



== Preliminary information ===

Much of this homework beyond this point will focus on the Apriori algorithm for frequent itemset  mining and the additional step for extracting association rules from these frequent itemsets.
Please acquaint yourself with the background information (below)
before approaching the remaining  assignments.

=== Apriori background information ===

Some background material for the  Apriori algorithm is located at:

 - Slides in Live Session #3
 - https://en.wikipedia.org/wiki/Apriori_algorithm
 - https://www.dropbox.com/s/k2zm4otych279z2/Apriori-good-slides.pdf?dl=0
 - http://snap.stanford.edu/class/cs246-2014/slides/02-assocrules.pdf

Association Rules are frequently used for Market Basket Analysis (MBA) by retailers to
understand the purchase behavior of their customers. This information can be then used for
many different purposes such as cross-selling and up-selling of products, sales promotions,
loyalty programs, store design, discount plans and many others.
Evaluation of item sets: Once you have found the frequent itemsets of a dataset, you need
to choose a subset of them as your recommendations. Commonly used metrics for measuring
significance and interest for selecting rules for recommendations are: confidence; lift; and conviction.

## HW3.6
What is the Apriori algorithm? Describe an example use in your domain of expertise and what kind of . Define confidence and lift.

NOTE:
For the remaining homework use the online browsing behavior dataset located at (same dataset as used above): 

       https://www.dropbox.com/s/zlfyiwa70poqg74/ProductPurchaseData.txt?dl=0

Each line in this dataset represents a browsing session of a customer. 
On each line, each string of 8 characters represents the id of an item browsed during that session. 
The items are separated by spaces.

Here are the first few lines of the ProductPurchaseData 
FRO11987 ELE17451 ELE89019 SNA90258 GRO99222 
GRO99222 GRO12298 FRO12685 ELE91550 SNA11465 ELE26917 ELE52966 FRO90334 SNA30755 ELE17451 FRO84225 SNA80192 
ELE17451 GRO73461 DAI22896 SNA99873 FRO86643 
ELE17451 ELE37798 FRO86643 GRO56989 ELE23393 SNA11465 
ELE17451 SNA69641 FRO86643 FRO78087 SNA11465 GRO39357 ELE28573 ELE11375 DAI54444 

## HW3.7. Shopping Cart Analysis
Product Recommendations: The action or practice of selling additional products or services 
to existing customers is called cross-selling. Giving product recommendation is 
one of the examples of cross-selling that are frequently used by online retailers. 
One simple method to give product recommendations is to recommend products that are frequently
browsed together by the customers.

Suppose we want to recommend new products to the customer based on the products they
have already browsed on the online website. Write a program using the A-priori algorithm
to find products which are frequently browsed together. Fix the support to s = 100 
(i.e. product sets need to occur together at least 100 times to be considered frequent) 
and find itemsets of size 2 and 3.

Then extract association rules from these frequent items. 

A rule is of the form: 

(item1, item5) ⇒ item2.

List the top 10 discovered rules in descreasing order of confidence in the following format
 
(item1, item5) ⇒ item2, supportCount ,support, confidence

## HW3.8

Benchmark your results using the pyFIM implementation of the Apriori algorithm
(Apriori - Association Rule Induction / Frequent Item Set Mining implemented by Christian Borgelt). 
You can download pyFIM from here: 

http://www.borgelt.net/pyfim.html

Comment on the results from both implementations (your Hadoop MapReduce of apriori versus pyFIM) 
in terms of results and execution times.


HW3.8 (Conceptual Exercise)

Suppose that you wished to perform the Apriori algorithm once again,
though this time now with the goal of listing the top 5 rules with corresponding confidence scores 
in decreasing order of confidence score for itemsets of size 3 using Hadoop MapReduce.
A rule is now of the form: 

(item1, item2) ⇒ item3 

Recall that the Apriori algorithm is iterative for increasing itemset size,
working off of the frequent itemsets of the previous size to explore 
ONLY the NECESSARY subset of a large combinatorial space. 
Describe how you might design a framework to perform this exercise.

In particular, focus on the following:
  — map-reduce steps required
  - enumeration of item sets and filtering for frequent candidates


END OF HOMEWORK
===============